# Data Collection 

##Setup

I will use Youtube API to collect data

In [ ]:
import os
import re
import json
import operator
from datetime import datetime, timedelta
from apiclient import discovery
from apiclient.discovery import build
import googleapiclient.discovery
import googleapiclient.errors

import pandas as pd
#import googletrans
#from googletrans import Translator

#from google_trans_new import google_translator  

In [ ]:
youTubeApiKey=''
youtube=build('youtube','v3',developerKey=youTubeApiKey)
#Ryan
#channelId='UChGJGhZ9SOOHvBB0Y4DOO_w'
#NatGeo
#channelId='UCpVm7bg6pXKo1Pr6k5kxG9A'
#Bounce Patrol
#channelId='UC56cowXhoqRWHeqfSJkIQaA'
#Disney Junior
#channelId='UCNcdbMyA59zE-Vk668bKWOg'
#NBC Sport
channelId='UCqZQlzSHbVJrwrn5XvzrzcA'

In [ ]:
statdata=youtube.channels().list(part='statistics',id=channelId).execute()
stats=statdata['items'][0]['statistics']
stats

{'hiddenSubscriberCount': False,
 'subscriberCount': '1700000',
 'videoCount': '19705',
 'viewCount': '1108922071'}

In [ ]:
snippetdata=youtube.channels().list(part='snippet',id=channelId).execute()
snippetdata 

{'etag': 'jILKP0MAq4j7yOvOm6R2n8jJECc',
 'items': [{'etag': 'HUfjWxNkFwk2GJbVmfluC7mk6Jw',
   'id': 'UCqZQlzSHbVJrwrn5XvzrzcA',
   'kind': 'youtube#channel',
   'snippet': {'country': 'US',
    'customUrl': 'nbcsports',
    'description': 'NBC Sports Group serves sports fans 24/7 with premier live events, insightful studio shows, and compelling original programming. NBC Sports is an established leader in the sports media landscape with an unparalleled collection of sports properties that include the Olympics, NFL, Premier League, NHL, NASCAR, PGA TOUR, the Kentucky Derby, Tour de France, French Open, IndyCar and many more. Subscribe to our channel for the latest sporting news and highlights!\n\nVisit NBCSports.com: https://www.nbcsports.com\nSubscribe to our Motorsports on NBC YouTube channel: https://www.youtube.com/motorsportsonnbc\nFind NBC Sports on Facebook: https://www.facebook.com/NBCSports\nFollow NBC Sports on Twitter: https://twitter.com/nbcsports\nFollow NBC Sports on Instag

In [ ]:
title=snippetdata['items'][0]['snippet']['title']
title

'NBC Sports'

In [ ]:
description=snippetdata['items'][0]['snippet']['description']
description

'NBC Sports Group serves sports fans 24/7 with premier live events, insightful studio shows, and compelling original programming. NBC Sports is an established leader in the sports media landscape with an unparalleled collection of sports properties that include the Olympics, NFL, Premier League, NHL, NASCAR, PGA TOUR, the Kentucky Derby, Tour de France, French Open, IndyCar and many more. Subscribe to our channel for the latest sporting news and highlights!\n\nVisit NBCSports.com: https://www.nbcsports.com\nSubscribe to our Motorsports on NBC YouTube channel: https://www.youtube.com/motorsportsonnbc\nFind NBC Sports on Facebook: https://www.facebook.com/NBCSports\nFollow NBC Sports on Twitter: https://twitter.com/nbcsports\nFollow NBC Sports on Instagram: https://www.instagram.com/nbcsports/'

In [ ]:
contentdata=youtube.channels().list(id=channelId,part='contentDetails').execute()
playlist_id = contentdata['items'][0]['contentDetails']['relatedPlaylists']['uploads']
videos = [ ]
next_page_token = None

while 1:
    res = youtube.playlistItems().list(playlistId=playlist_id,
                                               part='snippet',
                                               maxResults=50,
                                               pageToken=next_page_token).execute()
    videos += res['items']
    next_page_token = res.get('nextPageToken')

    if next_page_token is None:
        break
print(videos[1])

{'kind': 'youtube#playlistItem', 'etag': 'a2EJIcpvXVl66AfkbAYBsXrbGts', 'id': 'VVVxWlFselNIYlZKcndybjVYdnpyemNBLjcwUE9BbnZuZmQw', 'snippet': {'publishedAt': '2021-07-19T00:45:19Z', 'channelId': 'UCqZQlzSHbVJrwrn5XvzrzcA', 'title': "U.S. women's basketball defeats Nigeria in final exhibition game before Olympics | NBC Sports", 'description': "The U.S. women's basketball team earns a dominant win over Nigeria, 93-62, to snap a two-game losing streak and finish their exhibition slate in Las Vegas, before heading to Tokyo. #NBCSports #Olympics #TokyoOlympics\n» Subscribe to NBC Sports: https://www.youtube.com/nbcsports?sub_confirmation=1\n» Watch Live Sports on NBCSports.com: http://www.nbcsports.com/live\n\nNBC Sports Group serves sports fans 24/7 with premier live events, insightful studio shows, and compelling original programming. NBC Sports is an established leader in the sports media landscape with an unparalleled collection of sports properties that include the Olympics, NFL, Premie

In [ ]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))
video_ids[:5]

['jLfqwuwuIK4', '70POAnvnfd0', 'IHB-2FqSb9Q', 'JsKvUng8YAk', 'ibrNSh3Xf3U']

In [ ]:
len(video_ids)

19705

In [ ]:
len_kids=1902
video_ids=video_ids[:len_kids]

In [ ]:
len(video_ids)

1902

In [ ]:
stats = []
snippetdata=[] 
for i in range(0, len(video_ids), 40):
    res = (youtube).videos().list(id=','.join(video_ids[i:i+40]),part='statistics').execute()
    stats += res['items']
    request = (youtube).videos().list(id=','.join(video_ids[i:i+40]),part='snippet').execute()
    snippetdata+= request['items']
        
print(stats[:5])

[{'kind': 'youtube#video', 'etag': 'qGDx35M7VE7Ezsc340cdQE6aS-I', 'id': 'jLfqwuwuIK4', 'statistics': {'viewCount': '3753', 'likeCount': '94', 'dislikeCount': '3', 'favoriteCount': '0', 'commentCount': '14'}}, {'kind': 'youtube#video', 'etag': 'ErHOx6JC54Js7L8KXWCSW1w8Djg', 'id': '70POAnvnfd0', 'statistics': {'viewCount': '5066', 'likeCount': '126', 'dislikeCount': '7', 'favoriteCount': '0', 'commentCount': '29'}}, {'kind': 'youtube#video', 'etag': 'Keju8WG9vSiozCwMpVbWpZR34Z0', 'id': 'IHB-2FqSb9Q', 'statistics': {'viewCount': '59507', 'likeCount': '860', 'dislikeCount': '17', 'favoriteCount': '0', 'commentCount': '127'}}, {'kind': 'youtube#video', 'etag': 'cZ57ETPgabz91rcuK3qZcRYtZz8', 'id': 'JsKvUng8YAk', 'statistics': {'viewCount': '18192', 'likeCount': '186', 'dislikeCount': '19', 'favoriteCount': '0', 'commentCount': '43'}}, {'kind': 'youtube#video', 'etag': '9yNfM90_hH75Ift65x4OdGIIdEs', 'id': 'ibrNSh3Xf3U', 'statistics': {'viewCount': '96102', 'likeCount': '633', 'dislikeCount': 

In [ ]:
print(snippetdata[:1])

[{'kind': 'youtube#video', 'etag': '8OTHNrUUQNco199qXAve3l9sfHo', 'id': 'jLfqwuwuIK4', 'snippet': {'publishedAt': '2021-07-19T05:00:26Z', 'channelId': 'UCqZQlzSHbVJrwrn5XvzrzcA', 'title': "U.S. men's basketball surges in second half to defeat Spain in final exhibition game | NBC Sports", 'description': "In a potential Gold Medal Match preview, the U.S. men's basketball team defeated Spain 83-76 in the Americans' final exhibition game before the Olympics. #NBCSports #Olympics #TokyoOlympics\n» Subscribe to NBC Sports: https://www.youtube.com/nbcsports?sub_confirmation=1\n» Watch Live Sports on NBCSports.com: http://www.nbcsports.com/live\n\nNBC Sports Group serves sports fans 24/7 with premier live events, insightful studio shows, and compelling original programming. NBC Sports is an established leader in the sports media landscape with an unparalleled collection of sports properties that include the Olympics, NFL, Premier League, NHL, NASCAR, PGA TOUR, the Kentucky Derby, Tour de Franc

In [ ]:
snippetdata[0]['snippet']

{'categoryId': '17',
 'channelId': 'UCqZQlzSHbVJrwrn5XvzrzcA',
 'channelTitle': 'NBC Sports',
 'defaultAudioLanguage': 'en',
 'description': "In a potential Gold Medal Match preview, the U.S. men's basketball team defeated Spain 83-76 in the Americans' final exhibition game before the Olympics. #NBCSports #Olympics #TokyoOlympics\n» Subscribe to NBC Sports: https://www.youtube.com/nbcsports?sub_confirmation=1\n» Watch Live Sports on NBCSports.com: http://www.nbcsports.com/live\n\nNBC Sports Group serves sports fans 24/7 with premier live events, insightful studio shows, and compelling original programming. NBC Sports is an established leader in the sports media landscape with an unparalleled collection of sports properties that include the Olympics, NFL, Premier League, NHL, NASCAR, PGA TOUR, the Kentucky Derby, Tour de France, French Open, IndyCar and many more. Subscribe to our channel for the latest sporting news and highlights!\n\nNBC Olympics is responsible for producing, programm

In [ ]:
title=[ ]
channel_title=[]
desc=[]
date=[]
tags=[]
liked=[ ]
disliked=[ ]
views=[ ]
#comment=[ ]

for i in range(len(video_ids)):
    title.append(snippetdata[i]['snippet'].get('title'))
    channel_title.append(snippetdata[i]['snippet'].get('channelTitle'))
    desc.append(snippetdata[i]['snippet'].get('description'))
    date.append(snippetdata[i]['snippet'].get('publishedAt'))
    tags.append(snippetdata[i]['snippet'].get('tags')) 
    liked.append(stats[i]['statistics'].get('likeCount'))
    disliked.append(stats[i]['statistics'].get('dislikeCount'))
    views.append(stats[i]['statistics'].get('viewCount'))
    #comment.append(statistics['items'][0]['statistics'].get('commentCount'))
    #comment.append(int((stats[i])['statistics']['commentCount']))

In [ ]:
data={'title':title,'channel_title':channel_title,'desc':desc,'date':date,'tags':tags,'liked':liked,'disliked':disliked,'views':views}
df=pd.DataFrame(data)
df.head(5)

,title,channel_title,desc,date,tags,liked,disliked,views
0,U.S. men's basketball surges in second half to...,NBC Sports,"In a potential Gold Medal Match preview, the U...",2021-07-19T05:00:26Z,"[Olympic Sports, NFL, NBA, MLB, NHL, PGA, Golf...",94,3,3753
1,U.S. women's basketball defeats Nigeria in fin...,NBC Sports,The U.S. women's basketball team earns a domin...,2021-07-19T00:45:18Z,"[Olympic Sports, NFL, NBA, MLB, NHL, PGA, Golf...",126,7,5066
2,Tour de France 2021: Stage 21 extended highlig...,NBC Sports,"The 2021 Tour de France concludes at Stage 21,...",2021-07-18T23:53:35Z,"[tour de france, Olympics, cycling, cycling ra...",860,17,59507
3,2021 Tour de France crash compilation | Cyclin...,NBC Sports,From an unaware fan to tumbling down a mountai...,2021-07-18T23:13:34Z,"[tour de france, Olympics, cycling, cycling ra...",186,19,18192
4,"Haskell Stakes 2021 ends in jockey fall, disqu...",NBC Sports,Watch the 2021 Haskell Stakes at Monmouth Park...,2021-07-17T23:44:19Z,"[Horse racing, monmouth park, monmouth 2021, f...",633,35,96102


In [ ]:
len(df)

1902

In [ ]:
#df.to_csv('data_like_nastya.csv',index=False, )
#df.to_csv('data_ryan.csv',index=False )
df.to_csv('data_sports.csv',index=False )
